In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import tensorflow_addons as tfa

from PIL import Image
from sklearn import metrics
from sklearn.model_selection import KFold, train_test_split
from tqdm.notebook import tqdm

from func import path, get_data, path_img
from func_keras import get_model, seed_everything
import tensorflow as tf
import efficientnet.keras as efn
import math
from sklearn.neighbors import NearestNeighbors

In [2]:
# input
df = get_data(path / 'train.csv', path_img)
df.head()

,posting_id,image,image_phash,title,label_group,filepath,target,title_edit
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_2278313361, train_129225211]",paper bag victoria secret
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3423213080, train_3386243561]","double tape 3m vhb 12 mm x 4,5 m original / do..."
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3803689425, train_2288590299]",maling tts canned pork luncheon meat 397 gr
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_3342059966, train_2406599165]",daster batik lengan pendek - motif acak / camp...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,C:\Users\PC\OneDrive - Seagroup\computer_vison...,"[train_921438619, train_3369186413]",nescafe \xc3\x89clair latte 220ml


In [3]:
train_filenames = tf.io.gfile.glob([str(path) + '/train_record/*.*'])
# test_filenames = tf.io.gfile.glob([str(path) + '/test_images/*.*'])

In [4]:
# Use correct image size with pretrained model
IMAGE_SIZE = (300, 300)
# Train-test-split size
TRAIN_SIZE = 0.8
# Initial learning rate
LR = 0.001
# ArcFace must assume a certain number of classes to optimize loss. May get better results on test set with higher N_CLASSES
N_CLASSES = df['label_group'].nunique()
AUTO = tf.data.experimental.AUTOTUNE
SEED = 42

EPOCHS = 30
BATCH_SIZE = 4
STEPS_PER_EPOCH = len(df) * TRAIN_SIZE // BATCH_SIZE
cosine_lr_fn = tf.keras.experimental.CosineDecay(LR, 3*STEPS_PER_EPOCH*10)

# Function to seed everything


In [5]:
# ArcFace parameters
# s: norm of input feature
# m: margin, Original paper states that m=0.5 gives best results. I found this variable to have the strongest effect when calculating final f1 score
params = {
    'm': 0.0001, 
    's': 15
}

In [6]:
# Function to augment data
# As data was serialized to TFRecords, I directly convert TFRecords to datasets and thus cannot use Keras ImageDataGenerator
def data_augment(posting_id, image, label_group, matches):   
    rotate = tf.random.uniform(shape=(), minval=-0.1*np.pi, maxval=0.1*np.pi)
    image = tfa.image.rotate(image, rotate, interpolation='bilinear', fill_mode='constant')
    shear_x = tf.random.uniform(shape=(), minval=-0.2, maxval=0.2)
    shear_y = tf.random.uniform(shape=(), minval=-0.2, maxval=0.2)
    image = tfa.image.transform(image, [1.0, shear_x, 0.0, shear_y, 1.0, 0.0, 0.0, 0.0], interpolation='bilinear', fill_mode='constant')
    translate_vec = tf.random.uniform(shape=(2,), minval=-int(0.05*IMAGE_SIZE[0]), maxval=int(0.05*IMAGE_SIZE[0]))
    image = tfa.image.translate(image, translate_vec, interpolation='bilinear', fill_mode='constant')
    
    crop_size = tf.random.uniform(shape=(), minval=int(0.8*IMAGE_SIZE[0]), maxval=int(1.2*IMAGE_SIZE[0]), dtype=tf.int32)
    image = tf.image.resize_with_crop_or_pad(image, crop_size, crop_size)  
    image = tf.image.resize(image, IMAGE_SIZE)
    
    image = tf.image.random_brightness(image, 0.10)
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.80, 1.20)
    image = tf.image.random_contrast(image, 0.80, 1.20)
    image = tf.image.random_flip_left_right(image)
    return posting_id, image, label_group, matches

# Function to decode images from serialized image data from TFRecords
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Function to read TFRecords
def read_tfrec(example):
    tfrec_format = {
        "posting_id": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "label_group": tf.io.FixedLenFeature([], tf.int64),
        "matches": tf.io.FixedLenFeature([], tf.string)
    }

    example = tf.io.parse_single_example(example, tfrec_format)
    posting_id = example['posting_id']
    image = decode_image(example['image'])
    label_group = tf.cast(example['label_group'], tf.int32)
    matches = example['matches']
    return posting_id, image, label_group, matches

# Function to create a dataset by reading TFRecords
def load_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    dataset = dataset.map(read_tfrec, num_parallel_calls = AUTO) 
    return dataset

# Function to reformat dataset for model
def arcface_format(posting_id, image, label_group, matches):
    return posting_id, {'image': image, 'label': label_group}, label_group, matches

# Function to construct dataset
def get_dataset(filenames, training=False):
    dataset = load_dataset(filenames)
    if training:
        ignore_order = tf.data.Options()
        dataset = dataset.with_options(ignore_order)
        dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
        dataset = dataset.repeat()
        dataset = dataset.shuffle(2048)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [7]:
# Split data into train and validation sets
train, valid = train_test_split(train_filenames, shuffle = True, random_state = SEED)
train_dataset = get_dataset(train, training=True)

In [8]:
model = get_model(params, N_CLASSES, IMAGE_SIZE, cosine_lr_fn)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 efficientnetb3 (Functional)    (None, None, None,   10783535    ['image[0][0]']                  
                                1536)                                                             
                                                                                                  
 ge_m_pooling_layer (GeMPooling  (None, 1536)        0           ['efficientnetb3[0][0]']         
 Layer)                                                                                       

In [9]:
seed = 42
seed_everything(seed)
train, valid = train_test_split(train_filenames, shuffle = True, random_state = seed)
train_dataset = get_dataset(train, training=True)
train_dataset = train_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
val_dataset = get_dataset(valid)
val_dataset = val_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
tf.keras.backend.clear_session()

model = get_model(params, N_CLASSES, IMAGE_SIZE, cosine_lr_fn)
# Model checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(f"EfficientNetB3_{IMAGE_SIZE[0]}_{SEED}_m{params['m']}_s{params['s']}.h5",
                                                monitor = 'val_loss',
                                                save_best_only = True,
                                                save_weights_only = True,
                                                mode = 'min')

history = model.fit(train_dataset,
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = EPOCHS,
                    callbacks = [checkpoint],
                    validation_data = val_dataset)

Epoch 1/30
6850/6850 [==============================] - 803s 114ms/step - loss: 10.4966 - sparse_categorical_accuracy: 1.8248e-04 - val_loss: 10.6343 - val_sparse_categorical_accuracy: 1.0951e-04
Epoch 2/30
6850/6850 [==============================] - 783s 114ms/step - loss: 9.8873 - sparse_categorical_accuracy: 2.1898e-04 - val_loss: 9.8636 - val_sparse_categorical_accuracy: 1.0951e-04
Epoch 3/30
6850/6850 [==============================] - 771s 113ms/step - loss: 9.5826 - sparse_categorical_accuracy: 4.7445e-04 - val_loss: 9.7930 - val_sparse_categorical_accuracy: 1.0951e-04
Epoch 4/30
6850/6850 [==============================] - 766s 112ms/step - loss: 9.3725 - sparse_categorical_accuracy: 0.0019 - val_loss: 9.7186 - val_sparse_categorical_accuracy: 4.3802e-04
Epoch 5/30
6850/6850 [==============================] - 774s 113ms/step - loss: 9.2725 - sparse_categorical_accuracy: 0.0015 - val_loss: 9.8342 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/30
6850/6850 [==============

KeyboardInterrupt: 